<a href="https://colab.research.google.com/github/shailymishra/Machine-Learning-Advanced/blob/main/Manifold_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment-2: Manifold Learning

The objective of this assignment is to apply the concepts you have learned in class on Manifold Learning and Spectral Clustering. 

## Instructions
  - For each question you need to write the sub-problem formulation in markdown. 
  - Ensure that this notebook runs without errors when the cells are run in sequence.
  - Plagiarism will not be tolerated.
  - Use only `python3` to run your code.
  - If you are facing issues running the notebook on your local system. Use google collab to run the notebook online. To run the notebook online, go to [google collab](!https://colab.research.google.com/notebooks/intro.ipynb). Go to `File  -> Upload Notebook` and import the notebook file.

__NOTE__: If you use online platforms, you will have to upload `swissroll.dat` file separately and change the path in the code cell which loads the data.

## Submission  
- Rename the notebook to `<roll_number>.ipynb` and submit **ONLY** the notebook file on moodle.

## Problems 
 - Question 1: Spectral Clustering (10 marks)
 - Question 2: Manifold Visualization (10 marks)
 - Question 3: Clustering and Visualizing high-dimensional data (10 marks)
 - Question 4: Classification (10 marks)

## Deadline 
The deadline of this assignment is 27th April, 2020, 11:59 PM

In [ ]:
import sys
# Install packages correctly
!{sys.executable} -m pip install numpy sklearn tensorflow keras
# Fix mpl version due to subtle API differences
!{sys.executable} -m pip install matplotlib==3.0.3

import numpy
# Allow usage of both `np` and `numpy`
np = numpy
import sklearn
import matplotlib

     |████████████████████████████████| 13.0MB 4.9MB/s 
ERROR: plotnine 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 3.0.3 which is incompatible.
ERROR: mizani 0.6.0 has requirement matplotlib>=3.1.1, but you'll have matplotlib 3.0.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: matplotlib 3.2.1
    Uninstalling matplotlib-3.2.1:
      Successfully uninstalled matplotlib-3.2.1


# Question 1: Spectral Clustering

Implement spectral clustering and evaluate on the given concentric circles dataset for this question.

## Part 1: Implementation
Implement spectral clustering function from scratch (for two clusters), taking as input the dataset. It must return the predicted clustering. Assume that the graph constructed is a fully connected graph. Use the normalized graph laplacian for this case.



In [ ]:
# Part 1: Spectral Clustering

import numpy
import scipy
from sklearn.cluster import KMeans
from scipy.optimize import minimize
from functools import partial

def spectral_clustering(X, n_clusters=2):
    """Args:
    X: numpy.array [num_samples, input_dim]
    
    Returns:
    Y_pred: numpy.array [num_samples]
      array of cluster labels
  """
  # Your code here
    def laplacian(A):
        """Computes the symetric normalized laplacian.
        L = D^{-1/2} A D{-1/2}
        """
        D = numpy.zeros(A.shape)
        w = numpy.sum(A, axis=0)
        D.flat[::len(w) + 1] = w ** (-0.5)  # set the diag of D to w
        return D.dot(A).dot(D)


    def k_means(X, n_clusters):
        kmeans = KMeans(n_clusters=n_clusters, random_state=1231)
        return kmeans.fit(X).labels_

    def automatic_prunning(X):
        D = X.shape[1]
        v = (D + 1) / 2


        def _log_sq(x, eps=1e-14):
            t = numpy.log(x + eps)
            return t * t

        def squared_exponential(x, y, sig=0.8, sig2=1):
            norm = numpy.linalg.norm(x - y)
            dist = norm * norm
            return numpy.exp(- dist / (2 * sig * sig2))

        def radial_basis_phi(x, y, b, v):
            norm = numpy.linalg.norm(x - y)
            return max(1.0 - norm / b, 0) ** v


        def com_aff_local_scaling(X):
            N = X.shape[0]
            ans = numpy.zeros((N, N))
            sig = []
            for i in range(N):
                dists = []
                for j in range(N):
                    dists.append(numpy.linalg.norm(X[i] - X[j]))
                dists.sort()
                sig.append(numpy.mean(dists[:5]))

            for i in range(N):
                for j in range(N):
                    ans[i][j] = squared_exponential(X[i], X[j], sig[i], sig[j])
            return ans


        def _auto_prunning_find_b(X, v):
            K = com_aff_local_scaling(X)

            def compute_affinity(X, kernel=squared_exponential):
                N = X.shape[0]
                ans = numpy.zeros((N, N))
                for i in range(N):
                    for j in range(N):
                        ans[i][j] = kernel(X[i], X[j])
                return ans



            def _auto_prunning_cost(X, K, b, v, gamma=0.5):
                kernel = partial(radial_basis_phi, b=b, v=v)
                K_bv = compute_affinity(X, kernel)
                num = numpy.linalg.norm(K_bv - K)
                den = numpy.sqrt(numpy.linalg.norm(K_bv) * numpy.linalg.norm(K))
                rho = _log_sq(num / den)
                n = X.shape[0]
                s = 1.0 - (numpy.count_nonzero(K_bv) / (n * n))
                s = _log_sq(s)
                return numpy.sqrt((1.0 - gamma) * rho + gamma * s)



            def cost_b(x):
                return -1 * _auto_prunning_cost(X, K, x, v)  # we need to maximize this function

            result = minimize(
                cost_b,
                [numpy.mean(K)],
                bounds=((0, None),),  # positive
                # options={'disp': True, 'maxiter': 100})
            )
            return result.x[0]


        b = _auto_prunning_find_b(X, v)
        N = X.shape[0]
        affinity = com_aff_local_scaling(X)
        ans = numpy.zeros((N, N))
        for i in range(N):
            for j in range(N):
                ans[i][j] = radial_basis_phi(X[i], X[j], b, v) * affinity[i][j]
        return ans

    
    affinity = automatic_prunning(X)
    L = laplacian(affinity)
    eig_val, eig_vect = scipy.sparse.linalg.eigs(L, n_clusters)
    X = eig_vect.real
    rows_norm = numpy.linalg.norm(X, axis=1, ord=2)
    Y = (X.T / rows_norm).T
    Y_pred = k_means(Y, n_clusters)
    return Y_pred



## Part 2: Clustering concentric circles
Perform spectral clustering on the concentric circles dataset. Visualize the result by plotting it on a 2-d graph. Use different colours for different clusters.

In [ ]:
# DO NOT EDIT

from sklearn.datasets import make_circles 

CX, CY = make_circles(n_samples=200, shuffle=True,noise=0.05, random_state=1337, factor=0.5)
# CX: input data points [n_samples, 2]
# CY: true clusters [n_samples]

In [ ]:
# Part 2: Perform Spectral Clustering on the concentric circles dataset
# Plot using colors from CY (true clusters) and CY_pred (predicted clusters)
# Code in this cell should plot 2 subplots (true labels and predicted labels)


# fig = plt.figure()
# plt.scatter(CX[:, 0], CX[:, 1],**colorize)

fig = plt.figure()
# Original Clusters
ax = fig.add_subplot(1, 2, 1)
colorize = dict(c=CY, cmap=plt.cm.get_cmap('rainbow', 4))
ax.scatter(CX[:, 0], CX[:, 1],**colorize)
ax.set_title('Original Cluster')


# Predicted CLuster
ax = fig.add_subplot(1, 2, 2)
Y = spectral_clustering(CX)
ax.scatter(CX[:, 0], CX[:, 1], **colorize)
ax.set_title('Predicted Cluster')

fig.tight_layout(pad=3.0)
plt.show()


## Part 3: Evaluate accuracy
Evaluate the accuracy of the clustering by comparing it with the true labels. Create two subplots (true vs predicted) with the color of each point showing the cluster label.

In [ ]:
# Part 3: Report the accuracy of clustering
accuracy = np.sum(Y==CY)
print('Accuracy is : ', accuracy/len(Y)*100 , ' %')



---



# Question 2: Manifold Visualization
Implement the various manifold learning methods and visualize the given datasets.


## Part 1: MDS
Implement Multi-Dimensional Scaling

In [ ]:
def normalize(X):
    """Args:
    X: numpy.array [n, n]
    Returns:
    X: numpy.array [n, n]
  """
    
    n_samples = X.shape[0]

    # Mean for each row/column
    meanrows = np.sum(X, axis=0) / n_samples
    meancols = (np.sum(X, axis=1)/n_samples)[:, np.newaxis]

    # Mean across all rows (entire matrix)
    meanall = meanrows.sum() / n_samples

    X -= meanrows
    X -= meancols
    X += meanall
    return X



# Part 1: MDS
def MDS(X, output_dim=2):
    """Args:
        X: numpy.array [n_samples, input_dim]
        k: number of nearest neighbours to construct the knn graph
        output_dim: dimension of output data

        Returns:
        Y: numpy.array [n_samples, output_dim]
    """
    
    cov = X.dot(X.T) ## nxn
    cov = center(cov)
    
    eig_val_cov, eig_vec_cov = np.linalg.eig(cov)
    eig_pairs = [(np.abs(eig_val_cov[i]), eig_vec_cov[:, i]) for i in range(len(eig_val_cov))]
    
    # Select output_dim eigenvectors with largest eigenvalues, obtain subspace transform matrix
    eig_pairs.sort(key=lambda x: x[0], reverse=True)
    eig_pairs = np.array(eig_pairs)
    Y = np.hstack([eig_pairs[i, 1].reshape(cov.shape[1], 1) for i in range(output_dim)])
    
    return Y

## Part 2: LLE
Implement Locally Linear Embedding function

In [ ]:
# Part 2: LLE


def distance_mat(X, n_neighbors=100):
    """Args :
    Compute the square distance matrix using Euclidean distance
    X: (n_samples, input_dim)
    n_neighbors: Number of nearest neighbors to consider, int
    return neighbors, sort_distances 
    """
    def dist(a, b):
        return np.sqrt(sum((a - b)**2))

    # Compute full distance matrix
    distances = np.array([[dist(p1, p2) for p2 in X] for p1 in X])

    # Keep only the 6 nearest neighbors, others set to 0 (= unreachable)
    neighbors = np.zeros_like(distances)
    sort_distances = np.argsort(distances, axis=1)[:, 1:n_neighbors+1]
    for k,i in enumerate(sort_distances):
        neighbors[k,i] = distances[k,i]
    return neighbors, sort_distances



def LLE(X, k, output_dim=2):
    """Args:
    X: numpy.array [n_samples, input_dim]
    k: number of nearest neighbours to construct the knn graph
    output_dim: dimension of output data

    Returns:
    Y: numpy.array [n_samples, output_dim]
    """
    # Compute the nearest neighbors
    _, neighbors_idx = distance_mat(X, k)

    n = X.shape[0]
    w = np.zeros((n, n))
    for i in range(n):
        # Center the neighbors matrix
        k_indexes = neighbors_idx[i, :]
        neighbors = X[k_indexes, :] - X[i, :]

        # Compute the corresponding gram matrix
        gram_inv = np.linalg.pinv(np.dot(neighbors, neighbors.T))

        # Setting the weight values according to the lagrangian
        lambda_par = 2/np.sum(gram_inv)
        w[i, k_indexes] = lambda_par*np.sum(gram_inv, axis=1)/2
    m = np.subtract(np.eye(n), w)
    values, u = np.linalg.eigh(np.dot(np.transpose(m), m))
    Y = u[:, 1:output_dim+1]
    return Y


## Part 3: ISOMAP
Implement Isomap Visualization  

In [ ]:
# Part 3: Isomap

def ISOMAP(X, k, output_dim=100):
    """Args:
    X: numpy.array [n_samples, input_dim]
    k: number of nearest neighbours to construct the knn graph
    output_dim: dimension of output data

    Returns:
    Y: numpy.array [n_samples, output_dim]
  """
    # Compute distance matrix
    distance_matrix, _ = distance_mat(X, k)

    # Compute shortest paths from distance matrix
    from sklearn.utils.graph import graph_shortest_path
    graph = graph_shortest_path(distance_matrix, directed=False)
    graph = -0.5 * (graph ** 2)
    graph = center(graph)
    # Return the MDS projection on the shortest paths graph
    eig_val_cov, eig_vec_cov = np.linalg.eig(graph)
    eig_pairs = [(np.abs(eig_val_cov[i]), eig_vec_cov[:, i]) for i in range(len(eig_val_cov))]
    
    # Select output_dim eigenvectors with largest eigenvalues, obtain subspace transform matrix
    eig_pairs.sort(key=lambda x: x[0], reverse=True)
    eig_pairs = np.array(eig_pairs)
    Y = np.hstack([eig_pairs[i, 1].reshape(graph.shape[1], 1) for i in range(output_dim)])
    return Y


## Part 3: Manifold Visualization
Visualize the S-shaped 3-d dataset using the MDS, ISOMAP, LLE

In [ ]:
# DO NOT EDIT

from sklearn import manifold, datasets

SX, St = datasets.make_s_curve(n_samples=1000, random_state=1337)
# SX: input data [n_samples, 3]
# St: univariate position along manifold [n_samples], use for coloring the plots

The code in the next cell should draw a single plot with the following subplots:
1. 3D S-shaped dataset
2. 2D Manifold learnt using MDS
3. 2D Manifold learnt using ISOMAP
4. 2D Manifold learnt using LLE

Use the `St` variable to color the points in your visualizations. Use a color spectrum, and the position along the manifold to assign the color.

In [ ]:
#  Visualization code here
%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
import numpy as np

'''
Generate data and plot it
'''
colorize = dict(c=St, cmap=plt.cm.get_cmap('rainbow', 4))

# Data for three-dimensional scattered points

fig = plt.figure()
# 3D S shaped dataset subplot
ax = fig.add_subplot(2, 2, 1, projection='3d')
ax.scatter3D(SX[:,0], SX[:,1], SX[:,2],**colorize);
ax.set_title('Original data')

# 2D Manifold learning using MDS subplot
ax = fig.add_subplot(2, 2, 2)
# ax.scatter3D(SX[:,0], SX[:,1], SX[:,2],**colorize);
out = MDS(SX)
ax.scatter(out[:, 0], out[:, 1], **colorize)
ax.set_title('MDS')


# 2D Manifold learnt using ISOMAP
ax = fig.add_subplot(2, 2,3)
out = ISOMAP(SX,100)
ax.scatter(out[:, 0], out[:, 1], **colorize)
ax.set_title('ISOMAP , neighbours = 100')


# third subplot
ax = fig.add_subplot(2, 2, 4)
out = LLE(SX,100)
ax.scatter(out[:, 0], out[:, 1], **colorize)
ax.set_title('LLE , neighbours = 100')



fig.tight_layout(pad=3.0)
plt.show()


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


# Question 3: Clustering and Visualizing high-dimensional data
Perform k-means and spectral clustering on the Swiss roll dataset and visualize using the above 3 methods. State your observations.

In [ ]:
# Swiss roll dataset loading here
d = []
with open('./swissroll.dat', 'r') as dat_file:
    for line in dat_file:
        line = line.strip().split()
        line = [float(x.strip()) for x in line]
        d.append(line)
swissroll = numpy.array(d)
print (swissroll.shape)

(1600, 3)


Procedure for this question:
1. Perform spectral clustering (2 clusters) on the unchanged Swiss roll and visualize (binary colors)
2. Unwrap the manifold in 2D and visualize using
  - MDS
  - ISOMAP
  - LLE

Use the labels from the spectral clustering to color the unwrapped manifolds.

In [ ]:
# CODE HERE
%matplotlib notebook
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from sklearn.cluster import KMeans

## Kmeans and Spectral Clustering
fig = plt.figure()
# 3D S shaped dataset subplot
ax = fig.add_subplot(1, 2, 1, projection='3d')
Ylabels = spectral_clustering(swissroll)
colorize = dict(c=Ylabels, cmap=plt.cm.get_cmap('rainbow', 4))
ax.scatter3D(swissroll[:,0], swissroll[:,1], swissroll[:,2], **colorize);

ax = fig.add_subplot(1,2,2, projection='3d')
kmeans = KMeans(n_clusters=2, random_state=1231)
ylabels = kmeans.fit(swissroll).labels_
colorize = dict(c=ylabels, cmap=plt.cm.get_cmap('rainbow', 4))
ax.scatter3D(swissroll[:,0], swissroll[:,1], swissroll[:,2], **colorize);

plt.show()



colorize = dict(c=ylabels, cmap=plt.cm.get_cmap('rainbow', 4))

# Data for three-dimensional scattered points

fig = plt.figure()
# 2D Manifold learning using MDS subplot
ax = fig.add_subplot(2, 2, 1)
# ax.scatter3D(SX[:,0], SX[:,1], SX[:,2],**colorize);
out = MDS(swissroll)
ax.scatter(out[:, 0], out[:, 1], **colorize)

# 2D Manifold learnt using ISOMAP
ax = fig.add_subplot(2, 2,2)
out = ISOMAP(swissroll,100)
ax.scatter(out[:, 0], out[:, 1], **colorize)

# third subplot
ax = fig.add_subplot(2, 2, 3)
out = LLE(swissroll,100)
ax.scatter(out[:, 0], out[:, 1], **colorize)


plt.show()



As seen from the empirical data, spectral clustering finds cluster according to the distance on the manifold, while kmeans finds cluster just by euclidean distance in space.


---



# Question 4: Classification

Perform classification using a machine learning algorithm of your choice. Use 6k images from CIFAR-10 dataset.(5k images for training and 1k images for testing.)


*   Do dimensionality reduction on the dataset using PCA and ISOMAP.
*   Apply the classification algorithm.
*   Compare the results by changing the dimensionality of the data.
*   Use F1-score as metric.
*   Approach: Reduce the dimensionality into any two dimensions(of your choice) which are less than the initial dimensionality of the data using PCA and ISOMAP. Compare the performance metrics(F1-score) for the low dimensional data.



In [ ]:
# Code for loading CIFAR-10 dataset.
from keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
X_train = X_train[:5000].reshape([5000,32*32*3])
y_train = y_train[:5000]
X_test = X_test[:1000].reshape([1000,32*32*3])
y_test = y_test[:1000]
# Initial dimensionality/number of features (32*32*3) = 3072.

170500096/170498071 [==============================] - 4s 0us/step


In [ ]:
def reductionUsingPCA(X, output_dim):
  """Args:
    X: numpy.array [n_samples, input_dim]
    output_dim: dimension of output data

    Returns:
    pca_X: numpy.array [n_samples, output_dim]
  """
  # Enter your code here
  cov_mat = np.cov(X.T)
  eig_val_cov, eig_vec_cov = np.linalg.eig(cov_mat)
  eig_pairs = [(np.abs(eig_val_cov[i]), eig_vec_cov[:, i]) for i in range(len(eig_val_cov))]
  eig_pairs.sort(key=lambda x: x[0], reverse=True)
  eig_pairs = np.array(eig_pairs)
  matrix_w = np.hstack([eig_pairs[i, 1].reshape(X.shape[1], 1) for i in range(output_dim)])
  pca_X = np.dot(X, matrix_w)
  return pca_X


In [ ]:
def reductionUsingISOMAP(X, k, output_dim):
  """Args:
    X: numpy.array [n_samples, input_dim]
    k: number of nearest neighbours to construct the knn graph
    output_dim: dimension of output data

    Returns:
    isomap_X: numpy.array [n_samples, output_dim]
  """
  # Enter your code here
  def distance_mat(X, n_neighbors=100):
    """Args :
    Compute the square distance matrix using Euclidean distance
    X: (n_samples, input_dim)
    n_neighbors: Number of nearest neighbors to consider, int
    return neighbors, sort_distances 
    """
    def dist(a, b):
        return np.sqrt(sum((a - b)**2))

    # Compute full distance matrix
    distances = np.array([[dist(p1, p2) for p2 in X] for p1 in X])

    # Keep only the 6 nearest neighbors, others set to 0 (= unreachable)
    neighbors = np.zeros_like(distances)
    sort_distances = np.argsort(distances, axis=1)[:, 1:n_neighbors+1]
    for k,i in enumerate(sort_distances):
        neighbors[k,i] = distances[k,i]
    return neighbors, sort_distances

  print('okay')
  # Compute distance matrix
  distance_matrix, _ = distance_mat(X, k)

  # Compute shortest paths from distance matrix
  from sklearn.utils.graph import graph_shortest_path
  graph = graph_shortest_path(distance_matrix, directed=False)
  graph = -0.5 * (graph ** 2)
  graph = center(graph)
  # Return the MDS projection on the shortest paths graph
  eig_val_cov, eig_vec_cov = np.linalg.eig(graph)
  eig_pairs = [(np.abs(eig_val_cov[i]), eig_vec_cov[:, i]) for i in range(len(eig_val_cov))]
    
  # Select output_dim eigenvectors with largest eigenvalues, obtain subspace transform matrix
  eig_pairs.sort(key=lambda x: x[0], reverse=True)
  eig_pairs = np.array(eig_pairs)
  isomap_X = np.hstack([eig_pairs[i, 1].reshape(graph.shape[1], 1) for i in range(output_dim)])
  return isomap_X
    

In [ ]:
# Classification Algorithm 
# Extra functions here

def classification(X_train, y_train, X_test):
  from sklearn import svm
  svc = svm.SVC()
  y_train = y_train.ravel()
  svc.fit(X_train, y_train)
  labels = svc.predict(X_test)
  return labels


In [ ]:
def F1_score(y_true, y_pred):
  """Args:
    y_true: numpy.array [n_samples] , ground truth value
    y_pred: numpy.array [n_samples] , predicted value by classifier
    
    Returns:
    score: float, f1-score
    
  """
  # Enter your code here
  from sklearn.metrics import f1_score
  score = f1_score(y_true, y_pred, average='macro')
  return score

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


y_test = y_test.ravel()
x_train_pca_100 = reductionUsingPCA(X_train,100)
x_test_pca_100 = reductionUsingPCA(X_test,100)
y_pred_svm = classification(x_train_pca_100,y_train,x_test_pca_100)
f1_score = F1_score(y_test, y_pred_svm)
print('PCA dim reduction ,100 componentns , F1 score', f1_score)


x_train_pca_50 = reductionUsingPCA(X_train,50)
x_test_pca_50 = reductionUsingPCA(X_test,50)
y_pred_svm = classification(x_train_pca_50,y_train,x_test_pca_50)
f1_score = F1_score(y_test, y_pred_svm)
print('PCA dim reduction ,  50 componentns , F1 score', f1_score)


x_train_isomap_50 = reductionUsingISOMAP(X_train,200,50)
x_test_isomap_50 = reductionUsingISOMAP(X_test,200,50)
y_pred_svm = classification(x_train_isomap_50,y_train,x_test_isomap_50)
f1_score = F1_score(y_test, y_pred_svm)
print('ISO map dim reduction , 200 neighbours, 50 componentns , F1 score', f1_score)


x_train_isomap_100 = reductionUsingISOMAP(X_train,200,100)
x_test_isomap_100 = reductionUsingISOMAP(X_test,200,100)
y_pred_svm = classification(x_train_isomap_100,y_train,x_test_isomap_100)
f1_score = F1_score(y_test, y_pred_svm)
print('ISO map dim reduction , 200 neighbours, 100 componentns , F1 score', f1_score)

PCA dim reduction ,100 componentns , F1 score 0.4181686764339683
PCA dim reduction ,  50 componentns , F1 score 0.4240674659202283
ISO map dim reduction , 200 neighbours, 50 componentns , F1 score 0.09643335830211028
ISO map dim reduction , 200 neighbours, 100 componentns , F1 score 0.10165399721186334


As we can see, PCA peforms better, much better. As PCA tries to maximize variations, the original data's variance is maintained
and its easier to classify
Then in ISOMAP